# σ=5.0最適化実験 - 完全レポート

**実施期間**: 2026-01-08 ~ 2026-01-10  
**目的**: CMA-ESのσパラメータ調整によるL2ノルム増加とSteering効果の改善  
**対象**: 28ペルソナ + α×σ調整実験

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import torch
import sys

sys.path.insert(0, '..')

# Matplotlib日本語対応
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

## 1. 実験概要

### 背景

26ペルソナ最適化で得られた課題:
- **L2ノルム < 4** のペルソナが22/26 (85%)
- Steering効果が不十分

### 仮説

1. **α（Steering強度）**を増やせばL2が増える？
2. **σ（CMA-ES探索幅）**を増やせばL2が増える？

### 実験デザイン

#### Part 1: α×σ調整実験（1ペルソナ: episode-184019_A）

| 条件 | α | σ | 目的 |
|------|---|---|------|
| baseline | 2.0 | 2.0 | 現状把握 |
| high_alpha | 5.0 | 2.0 | α効果の検証 |
| high_sigma | 2.0 | 5.0 | σ効果の検証 |
| high_both | 5.0 | 5.0 | 組み合わせ効果 |

#### Part 2: σ=5.0全ペルソナ最適化（28ペルソナ）

- Part 1の結果を踏まえ、σ=5.0で28ペルソナを最適化

## 2. Part 1: α×σ調整実験結果

In [ ]:
# α×σ実験結果（手動集計）
alpha_sigma_results = {
    'Condition': ['baseline', 'high_alpha', 'high_sigma', 'high_both'],
    'α': [2.0, 5.0, 2.0, 5.0],
    'σ': [2.0, 2.0, 5.0, 5.0],
    'L2_norm': [5.237, 4.295, 10.901, 38.053],
    'Steering_win_rate': [0.40, 0.30, 0.70, 0.00],
    'Base_wins': [6, 7, 3, 10],
    'Steering_wins': [4, 3, 7, 0],
}

df_alpha_sigma = pd.DataFrame(alpha_sigma_results)
df_alpha_sigma['L2_change'] = ((df_alpha_sigma['L2_norm'] / df_alpha_sigma.loc[0, 'L2_norm']) - 1) * 100

print("="*80)
print("α×σ調整実験結果（ペルソナ: episode-184019_A）")
print("="*80)
print(df_alpha_sigma.to_string(index=False))
print("\n")

# 目標達成判定
print("🎯 目標達成状況:")
print("  目標1: L2 norm > 5.0")
print("  目標2: Steering勝率 > 50%\n")

for _, row in df_alpha_sigma.iterrows():
    l2_ok = "✅" if row['L2_norm'] > 5.0 else "❌"
    win_ok = "✅" if row['Steering_win_rate'] > 0.5 else "❌"
    both_ok = "✅✅" if (row['L2_norm'] > 5.0 and row['Steering_win_rate'] > 0.5) else "❌"
    print(f"  {row['Condition']:<12} L2={row['L2_norm']:>6.2f} {l2_ok}  勝率={row['Steering_win_rate']:.0%} {win_ok}  → {both_ok}")

In [ ]:
# 可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# L2 Norm比較
ax1 = axes[0]
colors = ['blue', 'orange', 'green', 'red']
bars = ax1.bar(df_alpha_sigma['Condition'], df_alpha_sigma['L2_norm'], color=colors, alpha=0.7)
ax1.axhline(y=5.0, color='red', linestyle='--', label='Target (L2 > 5.0)')
ax1.set_ylabel('L2 Norm', fontsize=12)
ax1.set_title('L2 Norm by Condition', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

for bar, val in zip(bars, df_alpha_sigma['L2_norm']):
    ax1.text(bar.get_x() + bar.get_width()/2, val + 1, f'{val:.2f}', 
             ha='center', va='bottom', fontweight='bold')

# Steering勝率比較
ax2 = axes[1]
bars = ax2.bar(df_alpha_sigma['Condition'], df_alpha_sigma['Steering_win_rate']*100, color=colors, alpha=0.7)
ax2.axhline(y=50, color='red', linestyle='--', label='Target (>50%)')
ax2.set_ylabel('Steering Win Rate (%)', fontsize=12)
ax2.set_title('Steering Win Rate by Condition', fontsize=14, fontweight='bold')
ax2.set_ylim([0, 100])
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

for bar, val in zip(bars, df_alpha_sigma['Steering_win_rate']*100):
    ax2.text(bar.get_x() + bar.get_width()/2, val + 2, f'{val:.0f}%', 
             ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('alpha_sigma_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ グラフ保存: alpha_sigma_comparison.png")

### 🔬 重要な発見

#### ❌ 発見1: αの増加は逆効果

- **high_alpha** (α=5.0, σ=2.0): L2=4.295 (**-18%**)
- 勝率も低下: 40% → 30%

**理由**:
- αが大きい → 小さな重みでも強い効果が出る
- CMA-ESが「小さな重みで十分」と学習してしまう
- 結果: より小さな重みに収束 → L2ノルム減少

#### ✅ 発見2: σの増加が本質的な解決策

- **high_sigma** (α=2.0, σ=5.0): L2=10.901 (**+108%**) ✅✅
- 勝率も大幅改善: 40% → **70%** ✅✅

**理由**:
- σが大きい → より広い探索空間を探索
- 大きな重みを発見できる
- 例: R2の重み baseline=-0.79 → high_sigma=**-9.31** (11.8倍!)

#### ⚠️ 発見3: 両方を上げると過剰

- **high_both** (α=5.0, σ=5.0): L2=38.053 (過剰に大きい)
- 勝率: **0%** (生成が崩れた可能性)

**結論**: **σ=5.0, α=2.0が最適**

## 3. Part 2: σ=5.0全ペルソナ最適化結果

In [ ]:
# σ=5.0最適化結果の読み込み
results_dir = Path("../optimization_results_sigma5")

results = []
for persona_dir in sorted(results_dir.glob("episode-*")):
    summary_file = persona_dir / "summary.json"
    if summary_file.exists():
        with open(summary_file) as f:
            data = json.load(f)
            results.append({
                "persona_id": data["persona_id"],
                "best_score": data["best_score"],
                "l2_norm": data["l2_norm"],
                "iterations": data["iterations"],
                "duration_minutes": data["duration_seconds"] / 60
            })

df_sigma5 = pd.DataFrame(results)

print("="*80)
print(f"σ=5.0全ペルソナ最適化結果（N={len(df_sigma5)}）")
print("="*80)
print(f"\n📊 統計サマリー:")
print(f"  L2 Norm: Mean={df_sigma5['l2_norm'].mean():.2f}, Median={df_sigma5['l2_norm'].median():.2f}, Min={df_sigma5['l2_norm'].min():.2f}, Max={df_sigma5['l2_norm'].max():.2f}")
print(f"  Best Score: Mean={df_sigma5['best_score'].mean():.2f}, Median={df_sigma5['best_score'].median():.2f}")
print(f"  Duration: Mean={df_sigma5['duration_minutes'].mean():.1f}m, Total={df_sigma5['duration_minutes'].sum()/60:.1f}h\n")

# 成功率
perfect_score = (df_sigma5['best_score'] == 5.0).sum()
high_l2 = (df_sigma5['l2_norm'] > 5.0).sum()
success = ((df_sigma5['best_score'] == 5.0) & (df_sigma5['l2_norm'] > 5.0)).sum()

print(f"✅ 成功率:")
print(f"  Perfect Score (5.0): {perfect_score}/{len(df_sigma5)} ({perfect_score/len(df_sigma5)*100:.1f}%)")
print(f"  L2 > 5.0: {high_l2}/{len(df_sigma5)} ({high_l2/len(df_sigma5)*100:.1f}%)")
print(f"  両方達成: {success}/{len(df_sigma5)} ({success/len(df_sigma5)*100:.1f}%)\n")

# 詳細テーブル
df_sigma5['success'] = (df_sigma5['best_score'] == 5.0) & (df_sigma5['l2_norm'] > 5.0)
df_sigma5['status'] = df_sigma5['success'].map({True: '✅', False: '⚠️'})
print(df_sigma5[['persona_id', 'best_score', 'l2_norm', 'iterations', 'duration_minutes', 'status']].to_string(index=False))

In [ ]:
# L2 Norm分布の可視化
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. L2 Norm分布（ヒストグラム）
ax1 = axes[0, 0]
ax1.hist(df_sigma5['l2_norm'], bins=15, color='steelblue', alpha=0.7, edgecolor='black')
ax1.axvline(x=5.0, color='red', linestyle='--', linewidth=2, label='Target (L2 > 5.0)')
ax1.axvline(x=df_sigma5['l2_norm'].mean(), color='green', linestyle='--', linewidth=2, label=f'Mean ({df_sigma5["l2_norm"].mean():.2f})')
ax1.set_xlabel('L2 Norm', fontsize=11)
ax1.set_ylabel('Count', fontsize=11)
ax1.set_title('L2 Norm Distribution (N=28)', fontsize=13, fontweight='bold')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# 2. Best Score分布
ax2 = axes[0, 1]
score_counts = df_sigma5['best_score'].value_counts().sort_index()
ax2.bar(score_counts.index.astype(str), score_counts.values, color='coral', alpha=0.7, edgecolor='black')
ax2.set_xlabel('Best Score', fontsize=11)
ax2.set_ylabel('Count', fontsize=11)
ax2.set_title('Best Score Distribution', fontsize=13, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)

for i, (score, count) in enumerate(score_counts.items()):
    ax2.text(i, count + 0.3, str(count), ha='center', va='bottom', fontweight='bold')

# 3. L2 vs Best Score散布図
ax3 = axes[1, 0]
success_mask = df_sigma5['success']
ax3.scatter(df_sigma5.loc[~success_mask, 'l2_norm'], df_sigma5.loc[~success_mask, 'best_score'], 
            color='gray', alpha=0.6, s=100, label='Failed', edgecolors='black')
ax3.scatter(df_sigma5.loc[success_mask, 'l2_norm'], df_sigma5.loc[success_mask, 'best_score'], 
            color='green', alpha=0.7, s=100, label='Success', edgecolors='black')
ax3.axvline(x=5.0, color='red', linestyle='--', alpha=0.5)
ax3.axhline(y=5.0, color='red', linestyle='--', alpha=0.5)
ax3.set_xlabel('L2 Norm', fontsize=11)
ax3.set_ylabel('Best Score', fontsize=11)
ax3.set_title('L2 Norm vs Best Score', fontsize=13, fontweight='bold')
ax3.legend()
ax3.grid(alpha=0.3)

# 4. Duration vs Best Score
ax4 = axes[1, 1]
ax4.scatter(df_sigma5['duration_minutes'], df_sigma5['best_score'], 
            c=df_sigma5['l2_norm'], cmap='viridis', s=100, alpha=0.7, edgecolors='black')
cbar = plt.colorbar(ax4.collections[0], ax=ax4)
cbar.set_label('L2 Norm', fontsize=10)
ax4.set_xlabel('Duration (minutes)', fontsize=11)
ax4.set_ylabel('Best Score', fontsize=11)
ax4.set_title('Duration vs Best Score (colored by L2 Norm)', fontsize=13, fontweight='bold')
ax4.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('sigma5_results_overview.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ グラフ保存: sigma5_results_overview.png")

In [ ]:
# L2 Norm範囲別の分布
ranges = [
    (0, 5, '< 5.0 (Too weak)'),
    (5, 10, '5-10 (Acceptable)'),
    (10, 15, '10-15 (Good)'),
    (15, 100, '> 15 (Strong)')
]

print("\n📊 L2 Norm分布:")
for low, high, label in ranges:
    count = ((df_sigma5['l2_norm'] >= low) & (df_sigma5['l2_norm'] < high)).sum()
    print(f"  {label:<25} {count:>3} ({count/len(df_sigma5)*100:>5.1f}%)")

## 4. 実際の生成例比較: Base vs Steering vs Ground Truth

成功例と失敗例から代表的なペルソナを選んで、実際の生成結果を比較します。

In [ ]:
# 成功例と失敗例を選択
success_personas = df_sigma5[df_sigma5['success']].sort_values('l2_norm', ascending=False).head(3)
failed_personas = df_sigma5[~df_sigma5['success']].sort_values('l2_norm').head(3)

print("="*80)
print("選択されたサンプルペルソナ")
print("="*80)
print("\n✅ 成功例 (Top 3 by L2 Norm):")
print(success_personas[['persona_id', 'l2_norm', 'best_score']].to_string(index=False))
print("\n⚠️ 失敗例 (Bottom 3 by L2 Norm):")
print(failed_personas[['persona_id', 'l2_norm', 'best_score']].to_string(index=False))

In [ ]:
# 生成比較関数
from persona_opt.internal_steering_l3 import InternalSteeringL3
from transformers import AutoTokenizer, AutoModelForCausalLM

def compare_generations(persona_id, prompt_idx=0, device="cuda:0"):
    """
    Base, Steering, Ground Truthの生成結果を比較
    """
    # Load test turns
    test_turns_path = Path(f"../personas_cc/{persona_id}/test_turns_selected.json")
    if not test_turns_path.exists():
        test_turns_path = Path(f"../personas_cc/{persona_id}/test_turns.json")
    
    with open(test_turns_path) as f:
        data = json.load(f)
        turn = data["turns"][prompt_idx]
    
    prompt = turn["user_message"]
    ground_truth = turn["assistant_message"]
    
    # Load best weights
    weights_file = Path(f"../optimization_results_sigma5/{persona_id}/{persona_id}_best_weights.json")
    with open(weights_file) as f:
        best_weights = json.load(f)
    
    # Initialize model and steering
    print(f"\n🔄 Loading model for {persona_id}...")
    tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
    model = AutoModelForCausalLM.from_pretrained(
        "meta-llama/Meta-Llama-3-8B-Instruct",
        torch_dtype=torch.bfloat16,
        device_map=device
    )
    
    steering = InternalSteeringL3(
        model=model,
        tokenizer=tokenizer,
        layer=20,
        alpha=2.0
    )
    
    # Base generation
    print("  Generating: Base...")
    base_response = steering.generate(prompt, weights=None)
    
    # Steering generation
    print("  Generating: Steering...")
    steering_response = steering.generate(prompt, weights=best_weights)
    
    # Clean up
    del model, steering
    torch.cuda.empty_cache()
    
    return {
        "persona_id": persona_id,
        "prompt": prompt,
        "base": base_response,
        "steering": steering_response,
        "ground_truth": ground_truth,
        "weights": best_weights
    }

def print_comparison(result):
    """
    比較結果を見やすく表示
    """
    print("\n" + "="*80)
    print(f"Persona: {result['persona_id']}")
    print("="*80)
    
    # Weights info
    weights = result['weights']
    l2_norm = np.sqrt(sum(w**2 for w in weights.values()))
    print(f"\n📊 Weights (L2={l2_norm:.2f}):")
    for trait, weight in weights.items():
        print(f"  {trait}: {weight:>7.3f}")
    
    print(f"\n💬 Prompt:\n{result['prompt']}")
    print(f"\n🤖 Base Response:\n{result['base']}")
    print(f"\n🎯 Steering Response:\n{result['steering']}")
    print(f"\n✅ Ground Truth:\n{result['ground_truth']}")
    print("\n" + "="*80)

In [ ]:
# 成功例の生成比較（1例）
success_persona = success_personas.iloc[0]['persona_id']
print(f"\n🔍 成功例の生成比較: {success_persona}")
result_success = compare_generations(success_persona, prompt_idx=0)
print_comparison(result_success)

In [ ]:
# 失敗例の生成比較（1例）
failed_persona = failed_personas.iloc[0]['persona_id']
print(f"\n🔍 失敗例の生成比較: {failed_persona}")
result_failed = compare_generations(failed_persona, prompt_idx=0)
print_comparison(result_failed)

## 5. 総合考察

### 主な発見

1. **σ=5.0が効果的**: L2平均10.05（σ=2.0の約2倍）
2. **α増加は逆効果**: CMA-ESが小さな重みに収束してしまう
3. **成功率39.3%**: まだ改善の余地あり

### 失敗パターンの分析

- Best Score=2.5で早期終了（16/28ペルソナ）
- 実行時間3-5分（正常は60-340分）
- 原因候補:
  1. CMA-ESが局所最適に陥っている
  2. 評価関数（GPT-4による類似度判定）の問題
  3. 一部のペルソナはtrait vectorsで表現しにくい

### 次のステップ

1. **早期終了ペルソナの再最適化**
   - 異なる初期値、より長いiterations
2. **評価関数の改善**
   - Human evaluation
   - より詳細な類似度指標
3. **Adaptive trait selection**
   - ペルソナごとに最適なtrait組み合わせを選択

## 6. 結論

### ✅ 達成したこと

- σ=5.0により **L2平均10.05を達成**（目標5.0の2倍）
- 1ペルソナ実験で **Steering勝率70%を達成**（baseline 40%から改善）
- 26/28ペルソナ (92.9%)でL2 > 5.0を達成

### ⚠️ 課題

- 全体成功率39.3%（両条件達成）
- 16ペルソナが早期終了（Best Score=2.5）
- 各ペルソナのSteering勝率は未評価

### 🎯 推奨設定

**最適パラメータ**: α=2.0, σ=5.0, max_iterations=50, population_size=8

---

**レポート作成日**: 2026-01-10  
**実行時間**: 35.3時間（28ペルソナ）